In [100]:
# imports -- change ip and fingerprint between hosts
import pandas as pd
import pyexasol

host = '192.168.56.102/DC90C14858BD2A0A3FE10C5245C4032AF3E179041AB66687CF4E8430A7FC94EE:8563'   # Don't forget to change the fingerprint
user = 'sys'
password = 'exasol'

# Connect to Exasol
conn = pyexasol.connect(dsn=host, 
                        user=user, 
                        password=password, 
                        debug=False, 
                        protocol_version=pyexasol.PROTOCOL_V1)

In [95]:
q = '''
SELECT *
FROM AOL_SCHEMA.WEATHER_EVENTS
'''
    
weather = conn.export_to_pandas(q)
print(weather.shape, weather.columns, sep="\n")

(19882, 17)
Index(['BEGIN_DATE_TIME', 'END_DATE_TIME', 'EVENT_TYPE', 'BEGIN_DAY',
       'END_DAY', 'STATE_FIPS', 'INJURIES_DIRECT', 'INJURIES_INDIRECT',
       'DEATHS_DIRECT', 'DEATHS_INDIRECT', 'DAMAGE_PROPERTY', 'DAMAGE_CROPS',
       'EPISODE_ID', 'EVENT_ID', 'BEGIN_MONTH', 'END_MONTH', 'REGION'],
      dtype='object')


In [96]:
weather.head(3)

,BEGIN_DATE_TIME,END_DATE_TIME,EVENT_TYPE,BEGIN_DAY,END_DAY,STATE_FIPS,INJURIES_DIRECT,INJURIES_INDIRECT,DEATHS_DIRECT,DEATHS_INDIRECT,DAMAGE_PROPERTY,DAMAGE_CROPS,EPISODE_ID,EVENT_ID,BEGIN_MONTH,END_MONTH,REGION
0,2006-01-01 00:00:00.000000,2006-01-31 23:59:00.000000,Drought,1,31,8,0,0,0,0,1000,NaN,202408,5482479,1,1,Colorado
1,2006-01-01 00:00:00.000000,2006-01-31 23:59:00.000000,Drought,1,31,48,0,0,0,0,1000,NaN,203632,5486559,1,1,Texas
2,2006-01-01 00:00:00.000000,2006-01-31 23:59:00.000000,Drought,1,31,40,0,0,0,0,1000,NaN,203694,5486086,1,1,Oklahoma


In [90]:
column_to_exclude = 'EVENT_ID'

# Identify duplicates based on all columns except the specified one
duplicates_df = weather[weather.drop(columns=[column_to_exclude]).duplicated(keep=False)]

# Display the duplicates
print(duplicates_df)


Empty DataFrame
Columns: [BEGIN_DATE_TIME, END_DATE_TIME, EVENT_TYPE, BEGIN_DAY, END_DAY, STATE_FIPS, INJURIES_DIRECT, INJURIES_INDIRECT, DEATHS_DIRECT, DEATHS_INDIRECT, DAMAGE_PROPERTY, DAMAGE_CROPS, EPISODE_ID, EVENT_ID, BEGIN_MONTH, END_MONTH, REGION]
Index: []


In [ ]:
q2 = '''
SELECT QUERYDIM.QUERY
FROM AOL_SCHEMA.QUERYDIM
'''
    
query2 = conn.export_to_pandas(q2)

In [151]:
q = '''
WITH queries AS (
    SELECT DISTINCT
        TIMEDIM.[month],
        TIMEDIM.[day of the month],
        TIMEDIM.[hour],
        TIMEDIM.[minute],
        FACTS.ANONID,
        QUERYDIM.QUERY,
        FACTS.CLICK
    FROM AOL_SCHEMA.TIMEDIM
    LEFT JOIN AOL_SCHEMA.QUERYDIM ON QUERYDIM.ID = TIMEDIM.ID
    LEFT JOIN AOL_SCHEMA.FACTS ON FACTS.QUERYID = QUERYDIM.ID
    WHERE FACTS.CLICK = 1
)
SELECT *
FROM queries
'''
    
query = conn.export_to_pandas(q)
print(query.shape, query.columns, sep="\n")

(5514616, 7)
Index(['month', 'day of the month', 'hour', 'minute', 'ANONID', 'QUERY',
       'CLICK'],
      dtype='object')


In [152]:
query[query['QUERY']=='50kw frequency converter']

,month,day of the month,hour,minute,ANONID,QUERY,CLICK
393386,may,20,5,33,44971,50kw frequency converter,1


In [ ]:
query

No duplicated

In [160]:
Query='''
WITH queries AS (
    SELECT DISTINCT
        TIMEDIM.[month],
        TIMEDIM.[day of the month],
        TIMEDIM.[hour],
        TIMEDIM.[minute],
        QUERYDIM.QUERY,
        FACTS.ANONID,
        -- Precompute the formatted date as a timestamp
        TO_TIMESTAMP(
            '2006-' || 
            LPAD(
                CASE 
                    WHEN LOWER(TRIM(TIMEDIM.[month])) = 'january' THEN '01'
                    WHEN LOWER(TRIM(TIMEDIM.[month])) = 'february' THEN '02'
                    WHEN LOWER(TRIM(TIMEDIM.[month])) = 'march' THEN '03'
                    WHEN LOWER(TRIM(TIMEDIM.[month])) = 'april' THEN '04'
                    WHEN LOWER(TRIM(TIMEDIM.[month])) = 'may' THEN '05'
                    WHEN LOWER(TRIM(TIMEDIM.[month])) = 'june' THEN '06'
                    WHEN LOWER(TRIM(TIMEDIM.[month])) = 'july' THEN '07'
                    WHEN LOWER(TRIM(TIMEDIM.[month])) = 'august' THEN '08'
                    WHEN LOWER(TRIM(TIMEDIM.[month])) = 'september' THEN '09'
                    WHEN LOWER(TRIM(TIMEDIM.[month])) = 'october' THEN '10'
                    WHEN LOWER(TRIM(TIMEDIM.[month])) = 'november' THEN '11'
                    WHEN LOWER(TRIM(TIMEDIM.[month])) = 'december' THEN '12'
                    ELSE '00'
                END, 2, '0') || 
            '-' || 
            LPAD(CAST(TIMEDIM.[day of the month] AS VARCHAR(10)), 2, '0') || 
            ' ' ||
            LPAD(CAST(TIMEDIM.[hour] AS VARCHAR(2)), 2, '0') || ':' || 
            LPAD(CAST(TIMEDIM.[minute] AS VARCHAR(2)), 2, '0')
        ) AS formatted_date
    FROM AOL_SCHEMA.TIMEDIM
    LEFT JOIN AOL_SCHEMA.QUERYDIM ON QUERYDIM.ID = TIMEDIM.ID
    LEFT JOIN AOL_SCHEMA.FACTS ON FACTS.QUERYID = QUERYDIM.ID
    WHERE FACTS.CLICK = 1
    LIMIT 20
)
SELECT
    q.QUERY,
    we.EVENT_TYPE,
    COUNT(*) AS query_count
FROM queries q
LEFT JOIN AOL_SCHEMA.WEATHER_EVENTS we
    ON q.formatted_date BETWEEN we.BEGIN_DATE_TIME AND we.END_DATE_TIME
GROUP BY GROUPING SETS (
    (q.QUERY, we.EVENT_TYPE)
)
'''
weather_events = conn.export_to_pandas(Query)
weather_events.head(3)

ExaQueryError: 
(
    message     =>  syntax error, unexpected UNNEST_ [line 45, column 5] (Session: 1816213798902048315)
    dsn         =>  192.168.56.102/DC90C14858BD2A0A3FE10C5245C4032AF3E179041AB66687CF4E8430A7FC94EE:8563
    user        =>  sys
    schema      =>  
    session_id  =>  1816213798902048315
    code        =>  42000
    query       =>  EXPORT (
WITH queries AS (
    SELECT DISTINCT
        TIMEDIM.[month],
        TIMEDIM.[day of the month],
        TIMEDIM.[hour],
        TIMEDIM.[minute],
        QUERYDIM.QUERY,
        FACTS.ANONID,
        -- Precompute the formatted date as a timestamp
        TO_TIMESTAMP(
            '2006-' || 
            LPAD(
                CASE 
                    WHEN LOWER(TRIM(TIMEDIM.[month])) = 'january' THEN '01'
                    WHEN LOWER(TRIM(TIMEDIM.[month])) = 'february' THEN '02'
                    WHEN LOWER(TRIM(TIMEDIM.[month])) = 'march' THEN '03'
                    WHEN LOWER(TRIM(TIMEDIM.[month])) = 'april' THEN 

In [162]:
QUERY='''
WITH queries AS (
    SELECT DISTINCT
        TIMEDIM.[month],
        TIMEDIM.[day of the month],
        TIMEDIM.[hour],
        TIMEDIM.[minute],
        QUERYDIM.QUERY,
        FACTS.ANONID,
        -- Precompute the formatted date as a timestamp
        TO_TIMESTAMP(
            '2006-' || 
            LPAD(
                CASE 
                    WHEN LOWER(TRIM(TIMEDIM.[month])) = 'january' THEN '01'
                    WHEN LOWER(TRIM(TIMEDIM.[month])) = 'february' THEN '02'
                    WHEN LOWER(TRIM(TIMEDIM.[month])) = 'march' THEN '03'
                    WHEN LOWER(TRIM(TIMEDIM.[month])) = 'april' THEN '04'
                    WHEN LOWER(TRIM(TIMEDIM.[month])) = 'may' THEN '05'
                    WHEN LOWER(TRIM(TIMEDIM.[month])) = 'june' THEN '06'
                    WHEN LOWER(TRIM(TIMEDIM.[month])) = 'july' THEN '07'
                    WHEN LOWER(TRIM(TIMEDIM.[month])) = 'august' THEN '08'
                    WHEN LOWER(TRIM(TIMEDIM.[month])) = 'september' THEN '09'
                    WHEN LOWER(TRIM(TIMEDIM.[month])) = 'october' THEN '10'
                    WHEN LOWER(TRIM(TIMEDIM.[month])) = 'november' THEN '11'
                    WHEN LOWER(TRIM(TIMEDIM.[month])) = 'december' THEN '12'
                    ELSE '00'
                END, 2, '0') || 
            '-' || 
            LPAD(CAST(TIMEDIM.[day of the month] AS VARCHAR(10)), 2, '0') || 
            ' ' ||
            LPAD(CAST(TIMEDIM.[hour] AS VARCHAR(2)), 2, '0') || ':' || 
            LPAD(CAST(TIMEDIM.[minute] AS VARCHAR(2)), 2, '0')
        ) AS formatted_date
    FROM AOL_SCHEMA.TIMEDIM
    LEFT JOIN AOL_SCHEMA.QUERYDIM ON QUERYDIM.ID = TIMEDIM.ID
    LEFT JOIN AOL_SCHEMA.FACTS ON FACTS.QUERYID = QUERYDIM.ID
    WHERE FACTS.CLICK = 1
),
-- Split the query into words using simple substring functions
split_queries AS (
    SELECT
        q.QUERY,
        we.EVENT_TYPE,
        LOWER(SUBSTRING(q.QUERY, 1, POSITION(' ' IN q.QUERY) - 1)) AS keyword,
        SUBSTRING(q.QUERY FROM POSITION(' ' IN q.QUERY) + 1) AS remaining_query
    FROM queries q
    LEFT JOIN AOL_SCHEMA.WEATHER_EVENTS we
        ON q.formatted_date BETWEEN we.BEGIN_DATE_TIME AND we.END_DATE_TIME
    WHERE POSITION(' ' IN q.QUERY) > 0
),
-- Add more iterations as needed, for example:
split_queries_2 AS (
    SELECT
        sq.QUERY,
        sq.EVENT_TYPE,
        LOWER(SUBSTRING(sq.remaining_query, 1, POSITION(' ' IN sq.remaining_query) - 1)) AS keyword,
        SUBSTRING(sq.remaining_query FROM POSITION(' ' IN sq.remaining_query) + 1) AS remaining_query
    FROM split_queries sq
    WHERE POSITION(' ' IN sq.remaining_query) > 0
)
-- Final output: Count occurrences of each keyword for each event type
SELECT
    keyword,
    EVENT_TYPE,
    COUNT(*) AS keyword_count
FROM split_queries_2
GROUP BY GROUPING SETS (
    (keyword, EVENT_TYPE),    -- Group by keyword and event type
    (keyword),                -- Also group by keyword alone for overall keyword count
    (EVENT_TYPE)              -- Group by event type alone for overall event count
)
ORDER BY EVENT_TYPE, keyword_count DESC
'''
weather_events = conn.export_to_pandas(QUERY)
weather_events.head(3)

ExaQueryAbortError: 
(
    message     =>  Client requested execution abort (Session: 1816213798902048315)
    dsn         =>  192.168.56.102/DC90C14858BD2A0A3FE10C5245C4032AF3E179041AB66687CF4E8430A7FC94EE:8563
    user        =>  sys
    schema      =>  
    session_id  =>  1816213798902048315
    code        =>  R0003
    query       =>  EXPORT (
WITH queries AS (
    SELECT DISTINCT
        TIMEDIM.[month],
        TIMEDIM.[day of the month],
        TIMEDIM.[hour],
        TIMEDIM.[minute],
        QUERYDIM.QUERY,
        FACTS.ANONID,
        -- Precompute the formatted date as a timestamp
        TO_TIMESTAMP(
            '2006-' || 
            LPAD(
                CASE 
                    WHEN LOWER(TRIM(TIMEDIM.[month])) = 'january' THEN '01'
                    WHEN LOWER(TRIM(TIMEDIM.[month])) = 'february' THEN '02'
                    WHEN LOWER(TRIM(TIMEDIM.[month])) = 'march' THEN '03'
                    WHEN LOWER(TRIM(TIMEDIM.[month])) = 'april' THEN '04'
          

In [156]:
weather_events.shape

(102, 3)

In [157]:
np.unique(weather_events['QUERY'])

array(['baron cameron', 'bmw z4 m', 'daisuki.tsukimi.org',
       'every15minutes', 'facebook.com', 'floor registers', 'google.com',
       'grand royal solaris', 'ludacris', 'midget females xxx rated',
       'oven baked potato', 'paint creek rochester michigan',
       'positive correlation', 'sagamore', 'selling a tower site',
       'taiwan adoption', 'the gazette paper iowa', 'wedding cakes',
       'yarn', 'ymca'], dtype=object)

In [158]:
weather_events[weather_events['QUERY']=='oven baked potato']

,QUERY,EVENT_TYPE,QUERY_COUNT
4,oven baked potato,Flood,35
8,oven baked potato,Debris Flow,6
9,oven baked potato,Heavy Rain,3
28,oven baked potato,Drought,14
53,oven baked potato,Wildfire,2
65,oven baked potato,Flash Flood,1


In [159]:
np.unique(weather_events['EVENT_TYPE'])

array(['Blizzard', 'Debris Flow', 'Drought', 'Flash Flood', 'Flood',
       'Hail', 'Heat', 'Heavy Rain', 'Heavy Snow', 'High Surf',
       'High Wind', 'Strong Wind', 'Thunderstorm Wind', 'Wildfire',
       'Winter Storm', 'Winter Weather'], dtype=object)

In [142]:
weather_events.head(25)

,QUERY,EVENT_TYPE,REGION,BEGIN_DATE_TIME,END_DATE_TIME
0,ominglot,Drought,Texas,2006-03-01 00:00:00.000000,2006-03-31 23:59:00.000000
1,personalized ceramic mugs,Drought,Texas,2006-04-01 00:00:00.000000,2006-04-30 23:59:00.000000
2,what banks offer the highest interest rates fo...,Drought,Texas,2006-03-01 00:00:00.000000,2006-03-31 23:59:00.000000
3,dave's ribs,Drought,Texas,2006-03-01 00:00:00.000000,2006-03-31 23:59:00.000000
4,watering restrictions in pinellas county,Drought,Texas,2006-03-01 00:00:00.000000,2006-03-31 23:59:00.000000
5,plants database,Drought,Texas,2006-05-01 00:00:00.000000,2006-05-08 23:59:00.000000
6,plants database,Drought,Texas,2006-05-01 00:00:00.000000,2006-05-31 23:59:00.000000
7,www.borg-warner.com,Drought,Texas,2006-05-01 00:00:00.000000,2006-05-31 23:59:00.000000
8,stock transfer company,Hail,Texas,2006-04-23 17:05:00.000000,2006-04-24 17:05:00.000000
9,stock transfer company,Drought,Texas,2006-04-01 00:00:00.000000,2006-04-30 23:59:00.000000


In [34]:
query='''
SELECT 
    QUERYDIM.ID AS QUERY_ID,
    QUERYDIM.QUERY AS FULL_QUERY,
    REGEXP_SUBSTR(QUERYDIM.QUERY, '[^ ]+', 1, seq.seq_num) AS KEYWORD,
    seq.seq_num AS WORD_POSITION
FROM AOL_SCHEMA.QUERYDIM
JOIN (
    SELECT LEVEL AS seq_num
    FROM DUAL
    CONNECT BY LEVEL <= 100 -- Adjust this to the max number of words in a query
) seq
ON REGEXP_SUBSTR(QUERYDIM.QUERY, '[^ ]+', 1, seq.seq_num) IS NOT NULL
WHERE QUERYDIM.ID IN (
    SELECT ID FROM AOL_SCHEMA.QUERYDIM WHERE ROWNUM <= 10
)
'''
weather_events = conn.export_to_pandas(query)
print(weather_events.head())

   QUERY_ID                    FULL_QUERY              KEYWORD  WORD_POSITION
0         1              donate cars cash               donate              1
1         2  cartwheels panthers allstars           cartwheels              1
2         3                        baders               baders              1
3         4                   toocool.com          toocool.com              1
4         5           wwwheartlandapp.com  wwwheartlandapp.com              1


In [36]:
weather_events

,QUERY_ID,FULL_QUERY,KEYWORD,WORD_POSITION
0,1,donate cars cash,donate,1
1,2,cartwheels panthers allstars,cartwheels,1
2,3,baders,baders,1
3,4,toocool.com,toocool.com,1
4,5,wwwheartlandapp.com,wwwheartlandapp.com,1
5,6,aa broward county meetings,aa,1
6,7,www.alpeha.com,www.alpeha.com,1
7,8,www.outriggeluanhotelwaikiki,www.outriggeluanhotelwaikiki,1
8,9,herman's bakery business,herman's,1
9,10,h3bo3 nah2bo3,h3bo3,1


In [32]:
weather_events.shape

(10, 4)

In [23]:
weather_events3[weather_events3['ANONID']==317691]

,QUERY,FORMATTED_DATE,ANONID,KEYWORD
0,cant stop scratching my vagina,2006-04-26,317691,cant
